In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
df_South_Korea= pd.read_csv("df_South_Korea.csv" ,encoding='unicode_escape')
df_South_Korea

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,South Korea,2020-01-01,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,South Korea,2020-01-02,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,South Korea,2020-01-03,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,South Korea,2020-01-04,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,South Korea,2020-01-05,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,South Korea,2021-09-24,51269185,82%,74836.0,13191185.0,1.459,257.112,24780.0,0.483,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008180
633,South Korea,2021-09-25,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008135
634,South Korea,2021-09-26,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008091
635,South Korea,2021-09-27,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008056


In [3]:
#shift 21days mortality
df_South_Korea_mortality=df_South_Korea["mortality"]
df_South_Korea_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
         ...   
632    0.008180
633    0.008135
634    0.008091
635    0.008056
636    0.000000
Name: mortality, Length: 615, dtype: float64

In [5]:
#删除后21天
df_South_Korea.drop(df_South_Korea.index[615:637], inplace=True)
df_South_Korea

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,South Korea,2020-01-01,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,South Korea,2020-01-02,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,South Korea,2020-01-03,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,South Korea,2020-01-04,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,South Korea,2020-01-05,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,South Korea,2021-09-02,51269185,82%,32409.0,12403576.0,0.632,241.761,35339.0,0.689,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008977
611,South Korea,2021-09-03,51269185,82%,57908.0,12461484.0,1.129,242.889,36024.0,0.702,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008941
612,South Korea,2021-09-04,51269185,82%,38342.0,12499826.0,0.747,243.637,37073.0,0.723,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008913
613,South Korea,2021-09-05,51269185,82%,8218.0,12508044.0,0.160,243.797,36387.0,0.709,...,0.0,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008889


In [6]:
df_South_Korea1=df_South_Korea.join(df_South_Korea_mortality,how="left",rsuffix="after21days")
df_South_Korea1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,South Korea,2020-01-01,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,South Korea,2020-01-02,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,South Korea,2020-01-03,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,South Korea,2020-01-04,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,South Korea,2020-01-05,51269185,82%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,South Korea,2021-09-02,51269185,82%,32409.0,12403576.0,0.632,241.761,35339.0,0.689,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008977,0.008977
611,South Korea,2021-09-03,51269185,82%,57908.0,12461484.0,1.129,242.889,36024.0,0.702,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008941,0.008941
612,South Korea,2021-09-04,51269185,82%,38342.0,12499826.0,0.747,243.637,37073.0,0.723,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008913,0.008913
613,South Korea,2021-09-05,51269185,82%,8218.0,12508044.0,0.160,243.797,36387.0,0.709,...,2.0,3.0,2.0,0.0,0.0,3.0,5.0,2.0,0.008889,0.008889


In [7]:
df_South_Korea1.to_csv("df_South_Korea_after21days",index=False)